# DSF e iESPnet

In [1]:
import sys
import os
import torch

import torchaudio.transforms    as T
import torch.optim              as optim
import pandas                   as pd
import numpy                    as np

from torchvision       import transforms
from torch.utils.data  import DataLoader
from torch             import nn
from sklearn.metrics   import balanced_accuracy_score, recall_score, precision_score, mean_absolute_error, average_precision_score


sys.path.append(os.path.abspath(os.path.join('..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTimev2, SeizureDatasetLabelTime, scale_spec, permute_spec, smoothing_label
from Model             import iESPnet
from TrainEval         import train_model_opt, test_model, train_model, get_thr_output, get_performance_indices
from IO                import get_spectrogram_2

sys.path.append(os.path.abspath(os.path.join('..','05-Train-Test')))
from utilit_train_test import make_weights_for_balanced_classes

sys.path.append(os.path.abspath(os.path.join('../../..','03 Dynamic-Spatial-Filtering')))
from models                         import DynamicSpatialFilter

In [2]:
# Variables iESPnet
FREQ_MASK_PARAM    = 10
TIME_MASK_PARAN    = 20
N_CLASSES          = 1
learning_rate      = 1e-3
batch_size         = 64    #128
epochs             = 20
num_workers        = 4

In [3]:
# Variables DSF
denoising          = 'autoreject'   # 'autoreject' 'data_augm' 
model              = 'stager_net'
dsf_type           = 'dsfd'         # 'dsfd' 'dsfm_st'
mlp_input          = 'log_diag_cov'
dsf_soft_thresh    = False
dsf_n_out_channels = None
n_channels         = 4

In [4]:
# hiperparametros iESPnet
hparams            = {
                      "n_cnn_layers" : 3,
                      "n_rnn_layers" : 3,
                      "rnn_dim"      : [150, 100, 50],
                      "n_class"      : N_CLASSES,
                      "out_ch"       : [8,8,16],
                      "dropout"      : 0.3,
                      "learning_rate": learning_rate,
                      "batch_size"   : batch_size,
                      "num_workers"  : num_workers,
                      "epochs"       : epochs
                     }

In [5]:
model1 = DynamicSpatialFilter(
                              n_channels, 
                              mlp_input            = mlp_input, 
                              n_out_channels       = dsf_n_out_channels, 
                              apply_soft_thresh    = dsf_soft_thresh
                             )

In [6]:
model2 = iESPnet(
                 hparams['n_cnn_layers'],
                 hparams['n_rnn_layers'],
                 hparams['rnn_dim'],
                 hparams['n_class'],
                 hparams['out_ch'],
                 hparams['dropout'],
                )

In [7]:
meta_data_iEEG = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'
model_path     = '/media/martin/Disco2/Rns_Data/models/models_DSF_iESPnet/PIT-RNS0427/models/model_opt.pth'

df_meta        = pd.read_csv(meta_data_iEEG)


In [8]:
len(df_meta['rns_id'].unique())

30

In [9]:
patients = df_meta['rns_id'].unique().tolist()

In [10]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

print('Using {} device'.format(device))

Using cuda device


In [11]:
kwargs = {'num_workers': hparams["num_workers"], 'pin_memory': True} if use_cuda else {}

In [12]:
checkpoint = torch.load(model_path)

In [13]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict1', 'model_state_dict2', 'optimizer_state_dict1', 'optimizer_state_dict2'])

In [14]:
model1.load_state_dict(checkpoint['model_state_dict1'])
model2.load_state_dict(checkpoint['model_state_dict2'])

<All keys matched successfully>

## Datos para testear

In [15]:
s = 28

test_df  = df_meta[df_meta['rns_id'] == patients[s]]

In [16]:
test_df

,rns_id,data,label,time
40336,PIT-RNS0427,PIT_RNS0427_20181120-1_E0,0,0.000
40337,PIT-RNS0427,PIT_RNS0427_20181120-1_E1,0,0.000
40338,PIT-RNS0427,PIT_RNS0427_20181120-1_E2,0,0.000
40339,PIT-RNS0427,PIT_RNS0427_20181120-1_E3,0,0.000
40340,PIT-RNS0427,PIT_RNS0427_20181120-1_E4,0,0.000
...,...,...,...,...
41916,PIT-RNS0427,PIT_RNS0427_20190723-1_E161,0,0.000
41917,PIT-RNS0427,PIT_RNS0427_20190723-1_E162,0,0.000
41918,PIT-RNS0427,PIT_RNS0427_20190723-1_E163,1,45.949
41919,PIT-RNS0427,PIT_RNS0427_20190723-1_E164,0,0.000


In [17]:
test_df.reset_index(drop=True, inplace=True)
test_df

,rns_id,data,label,time
0,PIT-RNS0427,PIT_RNS0427_20181120-1_E0,0,0.000
1,PIT-RNS0427,PIT_RNS0427_20181120-1_E1,0,0.000
2,PIT-RNS0427,PIT_RNS0427_20181120-1_E2,0,0.000
3,PIT-RNS0427,PIT_RNS0427_20181120-1_E3,0,0.000
4,PIT-RNS0427,PIT_RNS0427_20181120-1_E4,0,0.000
...,...,...,...,...
1580,PIT-RNS0427,PIT_RNS0427_20190723-1_E161,0,0.000
1581,PIT-RNS0427,PIT_RNS0427_20190723-1_E162,0,0.000
1582,PIT-RNS0427,PIT_RNS0427_20190723-1_E163,1,45.949
1583,PIT-RNS0427,PIT_RNS0427_20190723-1_E164,0,0.000


In [18]:
test_df = test_df[test_df['data'].str.contains('PIT_RNS0427_20190115-1')]
test_df

,rns_id,data,label,time
125,PIT-RNS0427,PIT_RNS0427_20190115-1_E0,0,0.0
126,PIT-RNS0427,PIT_RNS0427_20190115-1_E1,0,0.0
127,PIT-RNS0427,PIT_RNS0427_20190115-1_E2,0,0.0
128,PIT-RNS0427,PIT_RNS0427_20190115-1_E3,0,0.0
129,PIT-RNS0427,PIT_RNS0427_20190115-1_E4,0,0.0
...,...,...,...,...
316,PIT-RNS0427,PIT_RNS0427_20190115-1_E191,0,0.0
317,PIT-RNS0427,PIT_RNS0427_20190115-1_E192,0,0.0
318,PIT-RNS0427,PIT_RNS0427_20190115-1_E193,0,0.0
319,PIT-RNS0427,PIT_RNS0427_20190115-1_E194,0,0.0


In [19]:
test_df.reset_index(drop=True, inplace=True)
test_df.label.value_counts()

label
0    191
1      5
Name: count, dtype: int64

In [20]:
test_df[test_df['label'] == 1]

,rns_id,data,label,time
50,PIT-RNS0427,PIT_RNS0427_20190115-1_E50,1,1.012000
52,PIT-RNS0427,PIT_RNS0427_20190115-1_E52,1,15.477291
57,PIT-RNS0427,PIT_RNS0427_20190115-1_E57,1,48.497960
168,PIT-RNS0427,PIT_RNS0427_20190115-1_E168,1,48.450916
171,PIT-RNS0427,PIT_RNS0427_20190115-1_E171,1,47.961507


In [21]:
SPE_DIR = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'

In [22]:
# testing data should be balanced, just be "as it is"
test_data = SeizureDatasetLabelTimev2(
                                      file=test_df,
                                      root_dir=SPE_DIR,
                                      transform=None,
                                      target_transform=smoothing_label()  
                                     )

In [23]:
test_loader = DataLoader(test_data, batch_size = hparams['batch_size'], shuffle = False,**kwargs)

## Continuación

In [24]:
model1.to(device)

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [25]:
model2.to(device)

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [26]:
model1.eval()

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [27]:
model2.eval()

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

pensar en hacer un pdf con esto

## Guardado para pdf

In [28]:
pdf_path = '/media/martin/Disco2/Rns_Data/Representaciónes_PDF/'

if not os.path.exists(pdf_path):
    os.makedirs(pdf_path)

## Continuación

In [29]:
import matplotlib.pyplot as plt

In [30]:
# create spectrogram
ECOG_SAMPLE_RATE = 250
ECOG_CHANNELS    = 4
TT               = 1000 # window length
SPEC_WIN_LEN     = int(ECOG_SAMPLE_RATE * TT / 1000 ) # win size
overlap          = 500 
SPEC_HOP_LEN     = int(ECOG_SAMPLE_RATE * (TT - overlap) / 1000) # Length of hop between windows.
SPEC_NFFT        = 500  # to see changes in 0.5 reso
top_db           = 40.0

In [31]:
# Listas para almacenar los tensores en cada iteración
eeg_list            = []
labels_list         = []
outputs1_list       = []
spectrograms_list   = []
outputs2_list       = []

with torch.no_grad():
    for i, data_ in enumerate(test_loader):
            eeg, labels = data_
            eeg, labels = eeg.to(device), labels.to(device)

            outputs1 = model1(eeg)
            outputs1 = outputs1.squeeze(1)
            outputs1 = outputs1.to('cpu')
            
            spectrograms = get_spectrogram_2(outputs1, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)
            spectrograms = torch.from_numpy(spectrograms)
            spectrograms = spectrograms.to(device)
        
            outputs2 = model2(spectrograms)
            
            m     = nn.Sigmoid()
            probs = m(outputs2)

            # avoiding shape issues when the last batch has only one element
            if len(probs.shape) == 1:
                probs.unsqueeze_(0)
                outputs2.unsqueeze_(0)
            if i==0:
                y_true = labels
                y_probs = probs
                y_outputs = outputs2

            else:                
                y_true    = torch.cat((y_true, labels), axis=0)
                y_probs   = torch.cat((y_probs, probs), axis=0)
                y_outputs = torch.cat((y_outputs, outputs2), axis=0)
            
            eeg          = eeg.to('cpu')
            labels       = labels.to('cpu')
            outputs1     = outputs1.to('cpu')
            spectrograms = spectrograms.to('cpu')
            outputs2     = outputs2.to('cpu')

            eeg_list.append(eeg)
            labels_list.append(labels)
            outputs1_list.append(outputs1)
            spectrograms_list.append(spectrograms)
            outputs2_list.append(outputs2)

In [32]:
import matplotlib.font_manager as fm


sf               = 250    # frecuencia de muestreo
intervalo_tiempo = 1 / sf # segundos entre cada muestra
t_1              = np.arange(22500)
tiempo           = t_1 * intervalo_tiempo
ticks_x          = np.arange(0, tiempo.max()+1, 10)

## Modelo iESPnet

In [34]:
# Variables iESPnet
FREQ_MASK_PARAM    = 10
TIME_MASK_PARAN    = 20
N_CLASSES          = 1
learning_rate      = 1e-3
batch_size         = 64    #128
epochs             = 20
num_workers        = 4

In [35]:
# hiperparametros iESPnet
hparams            = {
                      "n_cnn_layers" : 3,
                      "n_rnn_layers" : 3,
                      "rnn_dim"      : [150, 100, 50],
                      "n_class"      : N_CLASSES,
                      "out_ch"       : [8,8,16],
                      "dropout"      : 0.3,
                      "learning_rate": learning_rate,
                      "batch_size"   : batch_size,
                      "num_workers"  : num_workers,
                      "epochs"       : epochs
                     }

In [36]:
model = iESPnet(
                hparams['n_cnn_layers'],
                hparams['n_rnn_layers'],
                hparams['rnn_dim'],
                hparams['n_class'],
                hparams['out_ch'],
                hparams['dropout'],
               )

In [37]:
meta_data_spec  = '/media/martin/Disco2/Rns_Data/PITT_PI_noprocess/METADATA_noprocess/allfiles_metadata.csv'
model_path_spec = '/media/martin/Disco2/Rns_Data/models/modelo-30-noprocess/PIT-RNS0427/models/model_opt.pth'

df_meta_spec    = pd.read_csv(meta_data_spec)

In [38]:
patients = df_meta_spec['rns_id'].unique().tolist()

In [39]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

print('Using {} device'.format(device))

Using cuda device


In [40]:
kwargs = {'num_workers': hparams["num_workers"], 'pin_memory': True} if use_cuda else {}

In [41]:
checkpoint = torch.load(model_path_spec)

In [42]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict'])

In [43]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [44]:
test_df_spec  = df_meta_spec[df_meta_spec['rns_id'] == patients[s]]

In [45]:
test_df_spec.reset_index(drop=True, inplace=True)
test_df_spec

,rns_id,data,label,time
0,PIT-RNS0427,PIT_RNS0427_20181120-1_E0,0,0.000
1,PIT-RNS0427,PIT_RNS0427_20181120-1_E1,0,0.000
2,PIT-RNS0427,PIT_RNS0427_20181120-1_E2,0,0.000
3,PIT-RNS0427,PIT_RNS0427_20181120-1_E3,0,0.000
4,PIT-RNS0427,PIT_RNS0427_20181120-1_E4,0,0.000
...,...,...,...,...
1580,PIT-RNS0427,PIT_RNS0427_20190723-1_E161,0,0.000
1581,PIT-RNS0427,PIT_RNS0427_20190723-1_E162,0,0.000
1582,PIT-RNS0427,PIT_RNS0427_20190723-1_E163,1,45.949
1583,PIT-RNS0427,PIT_RNS0427_20190723-1_E164,0,0.000


In [46]:
test_df_spec = test_df_spec[test_df_spec['data'].str.contains('PIT_RNS0427_20190115-1')]
test_df_spec.reset_index(drop=True, inplace=True)
test_df_spec

,rns_id,data,label,time
0,PIT-RNS0427,PIT_RNS0427_20190115-1_E0,0,0.0
1,PIT-RNS0427,PIT_RNS0427_20190115-1_E1,0,0.0
2,PIT-RNS0427,PIT_RNS0427_20190115-1_E2,0,0.0
3,PIT-RNS0427,PIT_RNS0427_20190115-1_E3,0,0.0
4,PIT-RNS0427,PIT_RNS0427_20190115-1_E4,0,0.0
...,...,...,...,...
191,PIT-RNS0427,PIT_RNS0427_20190115-1_E191,0,0.0
192,PIT-RNS0427,PIT_RNS0427_20190115-1_E192,0,0.0
193,PIT-RNS0427,PIT_RNS0427_20190115-1_E193,0,0.0
194,PIT-RNS0427,PIT_RNS0427_20190115-1_E194,0,0.0


In [47]:
label_1 = test_df_spec[test_df_spec['label'] == 1].index.tolist()
label_0 = test_df_spec[test_df_spec['label'] != 1].index.tolist()
indices = label_1 + label_0
print(indices)
test_df_spec_ordenado = test_df_spec.loc[indices].reset_index(drop=True)

[50, 52, 57, 168, 171, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195]


In [48]:
SPE_DIR = '/media/martin/Disco2/Rns_Data/PITT_PI_noprocess/'

In [49]:
# testing data should be balanced, just be "as it is"
test_data_spec = SeizureDatasetLabelTime(
                                      file=test_df_spec_ordenado,
                                      root_dir=SPE_DIR,
                                      transform=None,
                                      target_transform=smoothing_label()  
                                     )

In [50]:
test_loader_spec = DataLoader(test_data_spec, batch_size = hparams['batch_size'], shuffle = False,**kwargs)

In [51]:
model.to(device)

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [52]:
model.eval()

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [53]:
spec_list         = []
labels_list_spec  = []
outputs_list      = []


with torch.no_grad():
    for i, data_ in enumerate(test_loader_spec):
        spectrograms, labels = data_
        
        spectrograms, labels = spectrograms.to(device), labels.to(device)
        outputs = model(spectrograms)
        
        m = nn.Sigmoid()
        
        probs = m(outputs)
        # print(len(probs.shape))
        # avoiding shape issues when the last batch has only one element
        if len(probs.shape) == 1:
            probs.unsqueeze_(0)
            outputs.unsqueeze_(0)
        if i==0:
            y_true = labels
            y_probs = probs
            y_outputs = outputs

        else:                
            y_true = torch.cat((y_true, labels), axis=0)
            y_probs = torch.cat((y_probs, probs), axis=0)
            y_outputs = torch.cat((y_outputs, outputs), axis=0)
        
        spectrograms = spectrograms.to('cpu')
        labels       = labels.to('cpu')
        outputs      = outputs.to('cpu')

        spec_list.append(spectrograms)
        labels_list_spec.append(labels)
        outputs_list.append(outputs)

In [54]:
from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages(pdf_path + patients[s] + '.pdf', keep_empty=False)
font_path  = '../02-Letra-plot/Montserrat-Regular.ttf'
montserrat = fm.FontProperties(fname=font_path)

for i in range (len(eeg_list)):
    batch_size = eeg_list[i].size(0)
    for j in range (batch_size):
        fig, axs = plt.subplots(4, 1, figsize=(9, 9), facecolor='#F2F2F2')
        plt.subplots_adjust(left=0.15, right=0.95, top=0.95, bottom=0.1, hspace=0.4)
        
        axs[0].plot(tiempo, eeg_list[i][j, 3, :], color='#BEE6DC', linestyle='-', linewidth=1)
        axs[0].set_title('{} - {}'.format(patients[s], 'DSF input (iEEG)'), fontproperties=montserrat, fontsize=18)
        axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#CFCFCF')
        axs[0].set_xlim(0, 90)

        axs[0].axvline(x=test_df_spec_ordenado['time'][j], color='#8B97F4', linestyle='--', linewidth=1.5)

        axs[1].plot(tiempo, outputs1_list[i][j,3,:], color='#BEE6DC', linestyle='-', linewidth=1)
        axs[1].set_title('{} - {}'.format(patients[s], 'DSF output (iEEG)'), fontproperties=montserrat, fontsize=18)
        axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#CFCFCF')
        axs[1].set_xlim(0, 90)

        axs[2].imshow(spectrograms_list[i][j,3,:], origin='lower', aspect='auto')
        axs[2].set_title('{} - {}'.format(patients[s], 'iESPnet input (spectrogram)'), fontproperties=montserrat, fontsize=18)
    
        # Etiquetas del eje X para que correspondan a 0-90 segundos
        num_xticks = 10
        x_tick_positions = np.linspace(0, spectrograms_list[i][j,3,:].shape[1] - 1, num_xticks)
        x_tick_labels = [str(int(x *90 / (spectrograms_list[i][j,3,:].shape[1] - 1))) for x in x_tick_positions]
        axs[2].set_xticks(x_tick_positions)
        axs[2].set_xticklabels(x_tick_labels)

        # Etiquetas del eje Y para que correspondan a las frecuencias
        num_yticks = 6
        y_tick_positions = np.linspace(0, spectrograms_list[i][j,3,:].shape[0] - 1, num_yticks)
        y_tick_labels = [str(int(y * 60 / (spectrograms_list[i][j,3,:].shape[0] - 1))) for y in y_tick_positions]
        axs[2].set_yticks(y_tick_positions)
        axs[2].set_yticklabels(y_tick_labels)


        axs[3].plot(outputs2_list[i][j,:], color='#BEE6DC', linestyle='-', linewidth=1.5, label= 'DSF - iESPnet')
        axs[3].plot( outputs_list[i][j,:], color='#253D5B', linestyle='-', linewidth=1.5, label= 'iESPnet')
        axs[3].set_title('{} - {}'.format(patients[s], 'iESPnet output'), fontproperties=montserrat, fontsize=18)
        axs[3].set_xlabel('Tiempo (segundos)', fontsize=12, fontproperties=montserrat)
        axs[3].grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#CFCFCF')
        axs[3].set_xticks(x_tick_positions)
        axs[3].set_xticklabels(x_tick_labels)
        axs[3].set_xlim(0, 181)
        axs[3].legend()

        pp.savefig(fig)
        plt.close(fig)

pp.close()